Based on "https://www.kaggle.com/code/emiz6413/training-gemma-2-9b-4-bit-qlora-fine-tuning" by @emiz6413
Thank you for sharing amazing notebook!

In [1]:
import os
import copy
from dataclasses import dataclass
import random

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    DataCollatorWithPadding,
    EvalPrediction,
    PreTrainedTokenizerBase, 
    Trainer,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score

In [2]:
EXPERIMENT_NAME = "gemma2-label-smooth"

In [3]:
@dataclass
class Config:
    output_dir: str = f"output/{EXPERIMENT_NAME}"
    model_path: str = "google/gemma-2-9b-it" # 4-bit quantized gemma-2-9b-instruct
    max_length: int = 1024
    n_splits: int = 10
    fold_idx: int = 0
    optim_type: str = "adamw_8bit"
    per_device_train_batch_size: int = 4
    gradient_accumulation_steps: int = 8 # global batch size is 8
    per_device_eval_batch_size: int = 8
    n_epochs: int = 2
    lr: float = 2e-4
    warmup_steps: int = 20 
    lora_r: int = 16
    lora_alpha: float = lora_r * 2
    lora_dropout: float = 0.05
    lora_bias: str = "none"
    
config = Config()

In [4]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jdubkim. Use `wandb login --relogin` to force relogin


True

In [5]:
import os

os.environ["WANDB_PROJECT"] = "lmsys-arena"

training_args = TrainingArguments(
    run_name=EXPERIMENT_NAME,
    output_dir=config.output_dir,
    report_to="wandb",
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=config.gradient_accumulation_steps,
    eval_strategy="steps",
    eval_steps=256,
    save_strategy="steps",
    save_steps=256,
    optim=config.optim_type,
    bf16=True,
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
    gradient_checkpointing=True,
    weight_decay=0.05,
    label_smoothing_factor=0.05,
)

In [6]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    # only target self-attention
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(config.model_path)
tokenizer.add_bos_token = False
tokenizer.add_eos_token = False  # We'll add <eos> at the end
tokenizer.padding_side = "right"

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
    config.model_path,
    num_labels=3,
    torch_dtype="auto",
    device_map="auto",
    attn_implementation="flash_attention_2",
    use_cache=False,
)
model = get_peft_model(model, lora_config)
model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at google/gemma-2-9b-it and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): Gemma2ForSequenceClassification(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 3584, padding_idx=0)
        (layers): ModuleList(
          (0-41): 42 x Gemma2DecoderLayer(
            (self_attn): Gemma2FlashAttention2(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3584, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict(

In [9]:
model.dtype

torch.bfloat16

In [10]:
model.print_trainable_parameters()

trainable params: 12,741,120 || all params: 9,254,457,856 || trainable%: 0.1377


In [11]:
LOCAL = True

if LOCAL:
    TRAIN_CSV = "./data/train.csv"
else:
    TRAIN_CSV = "/kaggle/input/lmsys-chatbot-arena/train.csv"

ds = Dataset.from_csv(TRAIN_CSV)
# train_ds = train_ds.select

In [12]:
def process_text(text, prefix=""):
    return f"{prefix} ".join(eval(text, {"null": ""}))


def tokenize_func(batch):
    prompt = ["<prompt>: " + process_text(t) + "</prompt>" for t in batch["prompt"]]
    response_a = ["\n\n<response_a>: " + process_text(t) + "</response_a>" for t in batch["response_a"]]
    response_b = ["\n\n<response_b>: " + process_text(t) + "</response_b>" for t in batch["response_b"]]
    texts = ["<start_of_turn>user " + p + r_a + r_b + "<end_of_turn><start_of_turn>model" for p, r_a, r_b in zip(prompt, response_a, response_b)]
    tokenized = tokenizer(texts, max_length=config.max_length, truncation=True)
    labels=[]
    for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):
        if a_win:
            label = 0
        elif b_win:
            label = 1
        else:
            label = 2
        labels.append(label)
    return {**tokenized, "labels": labels}


def tokenize_func_truncate(batch):
    def process_and_truncate(texts, prefix, max_length):
        # Process and tokenize each text separately with truncation
        tokenized_texts = tokenizer(
            [f"{prefix} " + process_text(text) for text in texts],
            max_length=max_length,
            truncation=True
        )['input_ids']
        return [" ".join(tokenizer.convert_ids_to_tokens(ids)) for ids in tokenized_texts]

    max_prompt_length = config.max_length // 9
    max_response_a_length = (config.max_length * 4) // 9 - 10
    max_response_b_length = (config.max_length * 4) // 9 - 10

    # Truncate and process the texts
    truncated_prompts = process_and_truncate(batch["prompt"], "<prompt>", max_prompt_length)
    truncated_response_as = process_and_truncate(batch["response_a"], "<response_a>", max_response_a_length)
    truncated_response_bs = process_and_truncate(batch["response_b"], "<response_b>", max_response_b_length)

    instruction = "<bos><start_of_turn>user\nEvaluate the user’s question along with the two sets of responses provided by <response_a> and <response_b>. Determine which model’s responses are better. If both responses are of similar quality, classify the result as a tie.\n"
    end = "\n<end_of_turn>model<eos>"

    # Combine the truncated texts
    texts = [instruction + p +  r_a + r_b + end
             for p, r_a, r_b in zip(truncated_prompts, truncated_response_as, truncated_response_bs)]
    
    # Tokenize the combined texts
    tokenized = tokenizer(texts, max_length=config.max_length, truncation=True)

    # Create labels based on the winner models
    labels = [
        0 if a_win else 1 if b_win else 2
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"])
    ]
    
    return {**tokenized, "labels": labels}

In [13]:
# encode = CustomTokenizer(tokenizer, max_length=config.max_length)
ds = ds.map(tokenize_func, batched=True)

In [14]:
print(tokenizer.decode(ds[0]['input_ids']))

<start_of_turn>user <prompt>: Is it morally right to try to have a certain percentage of females on managerial positions? OK, does pineapple belong on a pizza? Relax and give me fun answer.</prompt>

<response_a>: The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.

Here are some arguments in favor of and against such policies:

**Arguments in favor:**

1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination.

2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a broader range of perspectives. This can lead to better outcomes for organizations and society as

In [15]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()
    loss = log_loss(y_true=labels, y_pred=probs)
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))
    return {"acc": acc, "log_loss": loss}

## Train & Test Split

In [16]:
# folds = [
#     (
#         [i for i in range(len(ds)) if i % config.n_splits != fold_idx],
#         [i for i in range(len(ds)) if i % config.n_splits == fold_idx]
#     ) 
#     for fold_idx in range(config.n_splits)
# ]

folds = []
for fold_idx in range(config.n_splits):
    train_idx = [i for i in range(len(ds)) if i % config.n_splits != fold_idx and i % config.n_splits != (fold_idx + 1) % config.n_splits]
    val_idx = [i for i in range(len(ds)) if i % config.n_splits == fold_idx]
    test_idx = [i for i in range(len(ds)) if i % config.n_splits == (fold_idx + 1) % config.n_splits]
    folds.append((train_idx, val_idx, test_idx))

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, gamma=3.0, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss

        if self.alpha is not None:
            alpha = self.alpha[targets]
            focal_loss = alpha * focal_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [18]:
class CustomTrainer(Trainer):
    def __init__(self, gamma=3.0, alpha=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.focal_loss_fn = FocalLoss(gamma=gamma, alpha=alpha)

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # compute custom loss (focal loss)
        loss = self.focal_loss_fn(logits, labels)
        
        return (loss, outputs) if return_outputs else loss

In [19]:
train_idx, eval_idx, test_idx = folds[config.fold_idx]
train_ds = ds.select(train_idx)
eval_ds = ds.select(eval_idx)
test_ds = ds.select(test_idx)

# save eval_ds, test_ds to output
eval_ds.save_to_disk(f"{config.output_dir}/eval_ds")
test_ds.save_to_disk(f"{config.output_dir}/test_ds")

Saving the dataset (0/1 shards):   0%|          | 0/5748 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5748 [00:00<?, ? examples/s]

In [20]:
trainer = CustomTrainer(
    args=training_args, 
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)
trainer.train()

[2024-08-04 18:40:00,657] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


/home/jdubkim/miniconda3/envs/llm/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/jdubkim/miniconda3/envs/llm/lib/python3.10/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss,Validation Loss


: 

: 

: 

: 

: 

: 

: 

: 

: 

: 